In [1]:
import geopandas as gpd
import os
from pprint import pprint
import numpy as np
from matplotlib import pyplot as plt
import rasterio as rio

# Import helper modules
import json
import mercantile
import requests
import urllib
import pandas as pd

from pl_b2p import *

In [2]:
# load the shapefile
rwanda_pts = gpd.read_file('../../rwanda/B2P_Rwanda_bridge_sites_2019_05_08.shp')
rwanda_pts.head()
print(rwanda_pts.Stage.unique())

rwanda_pts_NR = rwanda_pts[~rwanda_pts.Stage.isin(['Rejected', 'Identified'])].reset_index()
rwanda_pts_NR.shape, rwanda_pts.shape

['Rejected' 'Complete' 'Identified' 'Confirmed' 'Under Construction'
 'Prospecting']


((57, 21), (1521, 20))

In [3]:
rwanda_pts = gpd.read_file('../../rwanda/rwanda_no_pts_1000_minus_lakes.shp')

Get all of 'em

In [4]:
rwanda_poly = gpd.read_file(r"D:\projects\bridges_to_prosperity\Fwd__B2P_and_Earth_Lab_areas_of_interest\rwanda_aoi_big.geojson")
rwanda_poly = gpd.read_file(r"D:\projects\bridges_to_prosperity\Fwd__B2P_and_Earth_Lab_areas_of_interest\morija.geojson")
rwanda_bounds = list(rwanda_poly.bounds.values[0])
zm = 17 # for the bing images, SINCE WE CAN!
all_tiles_rwanda = list(mercantile.tiles(*rwanda_bounds, zooms=[17]))

In [5]:
from math import degrees, atan, sinh
import gdal

def x_to_lon_edges(x, z):
    tile_count = pow(2, z)
    unit = 360 / tile_count
    lon1 = -180 + x * unit
    lon2 = lon1 + unit
    return(lon1, lon2)

def mercatorToLat(mercatorY):
    return(degrees(atan(sinh(mercatorY))))


def y_to_lat_edges(y, z):
    tile_count = pow(2, z)
    unit = 1 / tile_count
    relative_y1 = y * unit
    relative_y2 = relative_y1 + unit
    lat1 = mercatorToLat(np.pi * (1 - 2 * relative_y1))
    lat2 = mercatorToLat(np.pi * (1 - 2 * relative_y2))
    return(lat1, lat2)

def tile_edges(x, y, z):
    lat1, lat2 = y_to_lat_edges(y, z)
    lon1, lon2 = x_to_lon_edges(x, z)
    return[lon1, lat1, lon2, lat2]

def georeference_raster_tile(x, y, z, path):
    bounds = tile_edges(x, y, z)
    filename, extension = os.path.splitext(path)
    gdal.Translate(filename + '.tif',
                   path,
                   outputSRS='EPSG:4326',
                   outputBounds=bounds)

In [12]:
_t = all_tiles_rwanda[0]
mercantile.ul(_t.x, _t.y, _t.z)

LngLat(lng=27.47955322265625, lat=-29.611670115197384)

In [15]:
# all_tiles_rwanda[0]
xp_ls, yp_ls, zp_ls, fp_ls = [],[],[],[]
fi_dir = 'morija_all_pngs_planet_256_2018'
os.makedirs(fi_dir, exist_ok=True)
for i, t in enumerate(all_tiles_rwanda):
    
    try:
        z = t.z
        x = t.x
        y = t.y
        t_url = f'https://tiles.planet.com/basemaps/v1/planet-tiles/{mosaic_name}/gmap/{z}/{x}/{y}.png?api_key={PLANET_API_KEY}'

        im_path = f'{fi_dir}/{x}_{y}_{z}.png'
        
        # skip if existing
        if os.path.exists(im_path):
            continue
        
        print(f'on {i+1} of {len(all_tiles_rwanda)}')
            
        urllib.request.urlretrieve(t_url, im_path)
        
        # georeference file
        georeference_raster_tile(x, y, z, im_path)

        xp_ls.append(x)
        yp_ls.append(y)
        zp_ls.append(z)
        fp_ls.append(im_path)
        
    except Exception as e:
        print(e)
        xp_ls.append(np.nan)
        yp_ls.append(np.nan)
        zp_ls.append(np.nan)
        fp_ls.append(np.nan)
        
    try:
        rc = 512

        lon = row[1]['geometry'].x
        lat = row[1]['geometry'].y
        zm = 17 # ~1.2m

        tile_mapping = mercantile.tile(lon, lat, zm)

        z=tile_mapping.z
        x=tile_mapping.x
        y=tile_mapping.y
        t_url = f'https://dev.virtualearth.net/REST/v1/Imagery/Map/Aerial/{lat},{lon}/{zm}?&format=png&mapSize={rc},{rc}&key=AgvMyZ32gp8jGrrVIN4I8tXRhfx3jmZKo0jgUaA9b1LHcq_yyGkmvGPS1nUXmzvs' 

        im_path = f'pngs_bing_256_no/rwanda_bing_{x}_{y}_{z}.png'
        urllib.request.urlretrieve(t_url, im_path)

        x_ls.append(x)
        y_ls.append(y)
        z_ls.append(z)
        f_ls.append(im_path)
        
    except Exception as e:
        print(e)
        x_ls.append(np.nan)
        y_ls.append(np.nan)
        z_ls.append(np.nan)
        f_ls.append(np.nan)
        

on 1 of 18
on 2 of 18
on 3 of 18
on 4 of 18
on 5 of 18
on 6 of 18
on 7 of 18
on 8 of 18
on 9 of 18
on 10 of 18
on 11 of 18
on 12 of 18
on 13 of 18
on 14 of 18
on 15 of 18
on 16 of 18
on 17 of 18
on 18 of 18


In [13]:
print('finished')

finished


In [10]:
xp_ls, yp_ls, zp_ls, fp_ls = [],[],[],[]

for row in rwanda_pts.iterrows():
    # specify a point from the geodataframe of accepted stages
    # get a tile coordiante for lon lat  (get from GDF)
    #pt_index=50
    #lon = rwanda_pts_NR.geometry[pt_index].x
    #lat = rwanda_pts_NR.geometry[pt_index].y
    
    try:
        lon = row[1]['geometry'].x
        lat = row[1]['geometry'].y
        zm = 15
        tile_mapping = mercantile.tile(lon, lat, zm)

        z=tile_mapping.z
        x=tile_mapping.x
        y=tile_mapping.y
        t_url = f'https://tiles.planet.com/basemaps/v1/planet-tiles/{mosaic_name}/gmap/{z}/{x}/{y}.png?api_key={PLANET_API_KEY}'

        im_path = f'pngs_planet_256_2018_no/{x}_{y}_{z}.png'
        
        # skip if existing
        if os.path.exists(im_path):
            continue
            
        urllib.request.urlretrieve(t_url, im_path)

        xp_ls.append(x)
        yp_ls.append(y)
        zp_ls.append(z)
        fp_ls.append(im_path)
        
    except Exception as e:
        print(e)
        xp_ls.append(np.nan)
        yp_ls.append(np.nan)
        zp_ls.append(np.nan)
        fp_ls.append(np.nan)
    

    

In [11]:
planet_df = pd.DataFrame({'tile_x':xp_ls,
                        'tile_y': yp_ls,
                       'tile_z': zp_ls,
                       'filename': fp_ls})

In [12]:
planet_df['filename'].unique().shape

(907,)

In [13]:
planet_df.to_csv('png_planet_256_2018_no.csv')

In [28]:
xp_ls, yp_ls, zp_ls, fp_ls = [],[],[],[]

# generate dataframe from 
for row in rwanda_pts.iterrows():
    
    try:
        lon = row[1]['geometry'].x
        lat = row[1]['geometry'].y
        zm = 15
        tile_mapping = mercantile.tile(lon, lat, zm)

        z=tile_mapping.z
        x=tile_mapping.x
        y=tile_mapping.y
        t_url = f'https://tiles.planet.com/basemaps/v1/planet-tiles/{mosaic_name}/gmap/{z}/{x}/{y}.png?api_key={PLANET_API_KEY}'

        im_path = f'pngs_planet_256/{x}_{y}_{z}.png'
                    
        #urllib.request.urlretrieve(t_url, im_path)

        xp_ls.append(x)
        yp_ls.append(y)
        zp_ls.append(z)
        fp_ls.append(im_path)
        
    except Exception as e:
        print(e)
        xp_ls.append(np.nan)
        yp_ls.append(np.nan)
        zp_ls.append(np.nan)
        fp_ls.append(np.nan)

cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer
cannot convert float NaN to integer


In [16]:
planet_df = pd.DataFrame({'tile_x':xp_ls,
                        'tile_y': yp_ls,
                       'tile_z': zp_ls,
                       'filename': fp_ls})

In [17]:
planet_df['filename'].unique().shape
planet_df.to_csv('png_planet_256_2018.csv')

(1253,)

# run for bing maps

In [14]:
from shapely.affinity import translate
from functools import partial

def add_coord_noise(df, maxdist=10):
    
    # add some spatial noise within specified limit
    x_shift = np.random.randn()*maxdist
    y_shift = np.random.randn()*maxdist
    
    # project the data and reproject back
    df = df.to_crs(epsg=3857)
    mapfunc = partial(translate, xoff=x_shift, yoff=y_shift)
    df['geometry'] = list(map(mapfunc, df['geometry']))
    df = df.to_crs(epsg=4326)
    
    return df

In [17]:
x_ls, y_ls, z_ls, f_ls = [],[],[],[]

# add spatial noise to each point, e.g., 10 meters
#rwanda_pts_coordnoise = add_coord_noise(rwanda_pts, 10)
rwanda_pts_coordnoise = rwanda_pts

# download tile for each point
for row in rwanda_pts_coordnoise.iterrows():
    # specify a point from the geodataframe of accepted stages
    # get a tile coordiante for lon lat  (get from GDF)
    #pt_index=50
    #lon = rwanda_pts_NR.geometry[pt_index].x
    #lat = rwanda_pts_NR.geometry[pt_index].y
    
    try:
        rc = 512

        lon = row[1]['geometry'].x
        lat = row[1]['geometry'].y
        zm = 17 # ~1.2m

        tile_mapping = mercantile.tile(lon, lat, zm)

        z=tile_mapping.z
        x=tile_mapping.x
        y=tile_mapping.y
        t_url = f'https://tiles.planet.com/basemaps/v1/planet-tiles/{mosaic_name}/gmap/{z}/{x}/{y}.png?api_key={PLANET_API_KEY}'
        t_url = f'https://dev.virtualearth.net/REST/v1/Imagery/Map/Aerial/{lat},{lon}/{zm}?&format=png&mapSize={rc},{rc}&key=AgvMyZ32gp8jGrrVIN4I8tXRhfx3jmZKo0jgUaA9b1LHcq_yyGkmvGPS1nUXmzvs' 

        im_path = f'pngs_bing_256_no/rwanda_bing_{x}_{y}_{z}.png'
        urllib.request.urlretrieve(t_url, im_path)

        x_ls.append(x)
        y_ls.append(y)
        z_ls.append(z)
        f_ls.append(im_path)
        
    except Exception as e:
        print(e)
        x_ls.append(np.nan)
        y_ls.append(np.nan)
        z_ls.append(np.nan)
        f_ls.append(np.nan)
        


<urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>
<urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>
<urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>
<urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>
<urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time,

In [18]:
bing_df = pd.DataFrame({'tile_x':x_ls,
                        'tile_y': y_ls,
                       'tile_z': z_ls,
                       'filename': f_ls})

In [19]:
bing_df['filename'].unique().shape

(987,)

Write data frames to csv

In [20]:
bing_df.to_csv('png_bing_256_no.csv')
